# ベストモデルの可視化（別ノート）

学習済みのベストモデル（`best_model_*.keras`）を読み込み、yfinanceで同じ手順でデータを取得したうえで、以下を表示します。
- ベストモデルで推論した結果（実測 vs 予測）
- Epoch と Loss の推移（`final_log_*.csv` から読み込み）

**前提**: `pso_lstm_5m.ipynb` でモデルを学習し、`best_model_*.keras` と `final_log_*.csv` が同じディレクトリに保存されていること。

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from pso_lstm_common import (
    add_technical_indicators,
    build_target_return,
    create_sequences,
    download_macro_daily,
    download_price_data,
    merge_macro_features,
    remove_high_corr_features,
    resample_ohlcv,
    resample_series,
    scale_train_val_test,
    train_val_test_split,
    wavelet_denoise,
)

# 日本語フォントの設定
try:
    import japanize_matplotlib

    print("japanize_matplotlib を使用して日本語フォントを設定しました")
except ImportError:
    # japanize_matplotlibが利用できない場合の代替設定
    plt.rcParams["font.family"] = [
        "DejaVu Sans",
        "Hiragino Sans",
        "Yu Gothic",
        "Meiryo",
        "Takao",
        "IPAexGothic",
        "IPAPGothic",
        "VL PGothic",
        "Noto Sans CJK JP",
    ]
    print("代替フォント設定を使用しました")
sns.set_theme(font="IPAexGothic")


warnings.filterwarnings("ignore")
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# メインノート（pso_lstm_5m.ipynb）と同じ設定
INDEX_TICKERS = ["^GSPC"]
FX_TICKER = "USDJPY=X"
RATE_TICKER = "^IRX"

BASE_INTERVAL = "5m"
BASE_PERIOD = "60d"

RESAMPLE_MINUTES = [5, 30, 60]
LOOKBACK_STEPS = {5: 5, 30: 30, 60: 60}

TRAIN_RATIO = 0.8
VAL_RATIO = 0.2

# 保存されたモデル・ログのファイル名パターン（同じディレクトリを想定）
MODEL_DIR = "."  # 必要に応じて変更

## 保存済みベストモデルで推論・可視化

In [ ]:
for ticker in INDEX_TICKERS:
    print(f"\n=== {ticker} ===")
    base_price = download_price_data(ticker, BASE_INTERVAL, BASE_PERIOD)

    try:
        fx_5m = download_price_data(FX_TICKER, BASE_INTERVAL, BASE_PERIOD)["close"]
    except Exception:
        fx_5m = download_macro_daily(FX_TICKER)

    rate_daily = download_macro_daily(RATE_TICKER)

    for minutes in RESAMPLE_MINUTES:
        model_path = os.path.join(MODEL_DIR, f"best_model_{ticker.replace('^', '')}_{minutes}m.keras")
        log_path = os.path.join(MODEL_DIR, f"final_log_{ticker.replace('^', '')}_{minutes}m.csv")
        if not os.path.isfile(model_path):
            print(f"スキップ: {model_path} がありません")
            continue

        print(f"--- {minutes}分足 ---")
        df = resample_ohlcv(base_price, minutes)
        df["close"] = wavelet_denoise(df["close"], level=3)
        df = add_technical_indicators(df)
        fx_resampled = resample_series(fx_5m, minutes)
        df = merge_macro_features(df, fx_resampled, rate_daily)
        df["target_return"] = build_target_return(df)
        df = df.dropna()

        feature_df, _ = remove_high_corr_features(df, target_col="target_return")
        feature_cols = [c for c in feature_df.columns if c != "target_return"]
        features = feature_df[feature_cols].values
        target = feature_df["target_return"].values

        lookback = LOOKBACK_STEPS[minutes]
        close_col_idx = feature_cols.index("close")
        X, y, last_close = create_sequences(features, target, lookback, close_col_idx=close_col_idx)
        (
            X_train,
            y_train,
            X_val,
            y_val,
            X_test,
            y_test,
            last_close_train,
            last_close_val,
            last_close_test,
        ) = train_val_test_split(
            X, y, train_ratio=TRAIN_RATIO, val_ratio=VAL_RATIO, last_close=last_close
        )
        X_train_s, X_val_s, X_test_s, y_train_s, y_val_s, y_test_s, x_scaler, y_scaler = scale_train_val_test(
            X_train, X_val, X_test, y_train, y_val, y_test
        )

        model = keras.models.load_model(model_path)
        pred_scaled = model.predict(X_test_s, verbose=0)
        pred_return = y_scaler.inverse_transform(pred_scaled).reshape(-1)
        # 騰落率→終値に復元（グラフは縦軸を終値で表示）
        y_pred_close = last_close_test * (1 + pred_return)
        y_true_close = last_close_test * (1 + y_test)

        # 推論結果（実測 vs 予測・縦軸は終値）
        fig, ax = plt.subplots(figsize=(10, 4))
        x_idx = np.arange(len(y_true_close))
        ax.plot(x_idx, y_true_close, label="実測", alpha=0.8)
        ax.plot(x_idx, y_pred_close, label="予測", alpha=0.8)
        ax.set_xlabel("サンプル番号")
        ax.set_ylabel("終値")
        ax.set_title(f"{ticker} {minutes}分足: ベストモデル推論結果（テストデータ）")
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

        # Epoch と Loss の推移（final_log_*.csv から）
        if os.path.isfile(log_path):
            log_df = pd.read_csv(log_path)
            fig, ax = plt.subplots(figsize=(8, 4))
            epochs_range = range(1, len(log_df) + 1)
            ax.plot(epochs_range, log_df["loss"], label="train loss")
            if "val_loss" in log_df.columns:
                ax.plot(epochs_range, log_df["val_loss"], label="val loss")
            ax.set_xlabel("Epoch")
            ax.set_ylabel("Loss (MSE)")
            ax.set_title(f"{ticker} {minutes}分足: 学習曲線")
            ax.legend()
            ax.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
        else:
            print(f"ログなし: {log_path}")

        tf.keras.backend.clear_session()